In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import time
import math
import community as community_louvain
import networkx.algorithms.community as modularity
from networkx.algorithms.community.quality import modularity
from collections import defaultdict


G = nx.read_edgelist("C:/Users/gnssl/Downloads/facebook_combined.txt", 
                     create_using=nx.Graph(), nodetype=int)

partition_louvain = community_louvain.best_partition(G)
len_partition = len(partition_louvain)

population_size = 200
generations = 1000
crossover_prob = 1.0
mutation_prob = 0.05
local_optimization_frequency=5

min_partition_size = 5
max_partition_size = 15

def initialize_population():
    population = []
    for idx in range(population_size):
        if idx%2==0:
            partition_louvain = community_louvain.best_partition(G)
            partition = [partition_louvain[node] for node in G.nodes]
            population.append(partition)
        else:
            partition = [random.randint(0, max_partition_size) for _ in range(len(G.nodes))]
            population.append(partition)
    return population

def compute_fitness(partition):
    communities = {key: value for key, value in zip(partition_louvain.keys(), partition)}
    new_dict = defaultdict(list)

    for key, value in communities.items():
        new_dict[value].append(key)

    result_dict = dict(new_dict)
    fitness = nx.community.modularity(G, result_dict.values())
    return fitness


def select_parents(population, fitness_values):
    parent1, parent2 = random.sample(population, 2)
    if compute_fitness(parent1) > compute_fitness(parent2):
        return parent1
    else:
        return parent2

def crossover(parent1, parent2):
    size = len(parent1)
    p1, p2 = [0] * size, [0] * size

    # Select two random points in the chromosomes
    point1 = random.randint(0, size - 1)
    point2 = random.randint(0, size - 1)

    # Ensure the two points are not the same
    while point1 == point2:
        point2 = random.randint(0, size - 1)

    # Order the points
    if point1 > point2:
        point1, point2 = point2, point1

    # Copy the respective parts from the parents to the children
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return child1, child2

def mutate(individual, mutation_prob, max_partition_size):
    rand_tmp = random.random()
    rand_idx = random.randint(1,len(individual))
    if rand_tmp < mutation_prob:
        for i in range(0,len(individual), rand_idx):
            # Mutate the community assignment of the node to a random community
            individual[i] = random.randint(0, max_partition_size - 1)
    return individual

def partition_to_communities(partition):
    communities = {}
    for node, community_id in enumerate(partition):
        if community_id in communities:
            communities[community_id].add(node)
        else:
            communities[community_id] = {node}
    return list(communities.values())



def genetic_algorithm():
    population = initialize_population()
    max_fitness_values = []  # Store max fitness values for each generation

    for generation in range(generations):
        fitness_values = [compute_fitness(partition) for partition in population]
        max_fitness = max(fitness_values)
        max_fitness_values.append(max_fitness)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        print('Generation', generation + 1, 'Max:', max_fitness)
        print('Generation', generation + 1, 'Avg:', avg_fitness)

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parents(population, fitness_values)
            parent2 = select_parents(population, fitness_values)
            if random.random() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                
                child1 = mutate(child1, mutation_prob, max_partition_size)
                child2 = mutate(child2, mutation_prob, max_partition_size)
                
                min_fitness_index = min(range(len(fitness_values)), key=fitness_values.__getitem__)
                population[min_fitness_index] = child1
                second_min_fitness_index = min(range(len(fitness_values)), key=lambda x: fitness_values[x] if x != min_fitness_index else float('inf'))
                population[second_min_fitness_index] = child2
                
        
        if generation % local_optimization_frequency == 0:
            random_idx = random.randint(1,population_size//2)
            for idx in range(0, random_idx, population_size//random_idx):
                partition_louvain = community_louvain.best_partition(G)
                partition = [partition_louvain[node] for node in G.nodes]
                population[idx] = partition
       
    # Select the best partition from the final population
    best_partition = max(population, key=compute_fitness)
    return best_partition, max_fitness_values

start = time.time()
best_partition, max_fitness_values = genetic_algorithm()

best_communities = partition_to_communities(best_partition)
end = time.time()
print('time:', end - start)

print("Best Partition:", best_communities)

Generation 1 Max: 0.8356572857172884
Generation 1 Avg: 0.4169483036616329
Generation 2 Max: 0.8356572857172884
Generation 2 Avg: 0.4248520870928189
Generation 3 Max: 0.8356572857172884
Generation 3 Avg: 0.43288033464551356
Generation 4 Max: 0.8356572857172884
Generation 4 Avg: 0.4400145193636373
Generation 5 Max: 0.8356572857172884
Generation 5 Avg: 0.4480473318222537
Generation 6 Max: 0.8356572857172884
Generation 6 Avg: 0.45579946664451426
Generation 7 Max: 0.8356572857172884
Generation 7 Avg: 0.4593924556688962
Generation 8 Max: 0.8356572857172884
Generation 8 Avg: 0.4677291291543406
Generation 9 Max: 0.8356572857172884
Generation 9 Avg: 0.476092753368154
Generation 10 Max: 0.8356572857172884
Generation 10 Avg: 0.48264569789033485
Generation 11 Max: 0.8356572857172884
Generation 11 Avg: 0.48957830968511173
Generation 12 Max: 0.8356572857172884
Generation 12 Avg: 0.4930966998187977
Generation 13 Max: 0.8356572857172884
Generation 13 Avg: 0.5000063115806775
Generation 14 Max: 0.835657

Generation 110 Max: 0.8358351159089359
Generation 110 Avg: 0.8248125824344339
Generation 111 Max: 0.8358351159089359
Generation 111 Avg: 0.8240573992439417
Generation 112 Max: 0.8358351159089359
Generation 112 Avg: 0.8253553275649854
Generation 113 Max: 0.8358351159089359
Generation 113 Avg: 0.8252007649041327
Generation 114 Max: 0.8358351159089359
Generation 114 Avg: 0.8257028018069172
Generation 115 Max: 0.8358351159089359
Generation 115 Avg: 0.8261759944305425
Generation 116 Max: 0.8358351159089359
Generation 116 Avg: 0.8254489146382705
Generation 117 Max: 0.8358351159089359
Generation 117 Avg: 0.8265322404169437
Generation 118 Max: 0.8358351159089359
Generation 118 Avg: 0.8265908308660355
Generation 119 Max: 0.8358351159089359
Generation 119 Avg: 0.8270263552903682
Generation 120 Max: 0.8358351159089359
Generation 120 Avg: 0.8271009963692371
Generation 121 Max: 0.8358351159089359
Generation 121 Avg: 0.8272388815229418
Generation 122 Max: 0.8358351159089359
Generation 122 Avg: 0.827

Generation 216 Max: 0.8357851325349899
Generation 216 Avg: 0.8322979843436029
Generation 217 Max: 0.8357283240010629
Generation 217 Avg: 0.8315352708153347
Generation 218 Max: 0.8357283240010629
Generation 218 Avg: 0.8314717357420374
Generation 219 Max: 0.8357283240010629
Generation 219 Avg: 0.8319423500243772
Generation 220 Max: 0.8357283240010629
Generation 220 Avg: 0.830796617877033
Generation 221 Max: 0.8357283240010629
Generation 221 Avg: 0.8317734396079837
Generation 222 Max: 0.8357283240010629
Generation 222 Avg: 0.8313195885344019
Generation 223 Max: 0.8357283240010629
Generation 223 Avg: 0.8324843523562581
Generation 224 Max: 0.8357283240010629
Generation 224 Avg: 0.8322326432457328
Generation 225 Max: 0.8357283240010629
Generation 225 Avg: 0.8325087868169037
Generation 226 Max: 0.8357283240010629
Generation 226 Avg: 0.832198742675272
Generation 227 Max: 0.8356572857172884
Generation 227 Avg: 0.8316732814918438
Generation 228 Max: 0.8356572857172884
Generation 228 Avg: 0.83217

Generation 322 Max: 0.835774575633323
Generation 322 Avg: 0.8329229306843626
Generation 323 Max: 0.835774575633323
Generation 323 Avg: 0.8328580818886708
Generation 324 Max: 0.835774575633323
Generation 324 Avg: 0.8340158345755172
Generation 325 Max: 0.835774575633323
Generation 325 Avg: 0.8345041725262411
Generation 326 Max: 0.835774575633323
Generation 326 Avg: 0.8343375108986192
Generation 327 Max: 0.835774575633323
Generation 327 Avg: 0.8340542723883293
Generation 328 Max: 0.835774575633323
Generation 328 Avg: 0.8343667395850644
Generation 329 Max: 0.835774575633323
Generation 329 Avg: 0.8345520090508066
Generation 330 Max: 0.835774575633323
Generation 330 Avg: 0.8341723196006633
Generation 331 Max: 0.835774575633323
Generation 331 Avg: 0.8346158107999331
Generation 332 Max: 0.835774575633323
Generation 332 Avg: 0.834498546051399
Generation 333 Max: 0.835774575633323
Generation 333 Avg: 0.8338809662788718
Generation 334 Max: 0.835774575633323
Generation 334 Avg: 0.8346237881394144


Generation 428 Max: 0.8358021283528634
Generation 428 Avg: 0.8340351128876158
Generation 429 Max: 0.8358021283528634
Generation 429 Avg: 0.8344795417549814
Generation 430 Max: 0.8358021283528634
Generation 430 Avg: 0.8345594966930391
Generation 431 Max: 0.8358021283528634
Generation 431 Avg: 0.8343994215570595
Generation 432 Max: 0.8358021283528634
Generation 432 Avg: 0.8335639543502912
Generation 433 Max: 0.8358021283528634
Generation 433 Avg: 0.8341701425617783
Generation 434 Max: 0.8358021283528634
Generation 434 Avg: 0.8336351003439928
Generation 435 Max: 0.8358021283528634
Generation 435 Avg: 0.834671335124369
Generation 436 Max: 0.8358021283528634
Generation 436 Avg: 0.8338294759338016
Generation 437 Max: 0.8358539153966056
Generation 437 Avg: 0.8344400789360329
Generation 438 Max: 0.8358539153966056
Generation 438 Avg: 0.8340318277231141
Generation 439 Max: 0.8358539153966056
Generation 439 Avg: 0.8342694788691465
Generation 440 Max: 0.8358539153966056
Generation 440 Avg: 0.8329

Generation 534 Max: 0.8357456762627253
Generation 534 Avg: 0.8334837666955426
Generation 535 Max: 0.8357456762627253
Generation 535 Avg: 0.8340579400327514
Generation 536 Max: 0.8357456762627253
Generation 536 Avg: 0.8330935357221945
Generation 537 Max: 0.8357456762627253
Generation 537 Avg: 0.8338942460812951
Generation 538 Max: 0.8357456762627253
Generation 538 Avg: 0.8342767430834205
Generation 539 Max: 0.8357456762627253
Generation 539 Avg: 0.8348344142212072
Generation 540 Max: 0.8357456762627253
Generation 540 Avg: 0.834077929208084
Generation 541 Max: 0.8357456762627253
Generation 541 Avg: 0.834495651558923
Generation 542 Max: 0.8357074444872683
Generation 542 Avg: 0.8338480218338727
Generation 543 Max: 0.8357074444872683
Generation 543 Avg: 0.8340367317605746
Generation 544 Max: 0.8357074444872683
Generation 544 Avg: 0.8348349141975244
Generation 545 Max: 0.8357074444872683
Generation 545 Avg: 0.8347762298370546
Generation 546 Max: 0.8357074444872683
Generation 546 Avg: 0.83438

Generation 640 Max: 0.8356572857172884
Generation 640 Avg: 0.8347213087722164
Generation 641 Max: 0.8356572857172884
Generation 641 Avg: 0.8345929577278467
Generation 642 Max: 0.8356572857172884
Generation 642 Avg: 0.8348243414138784
Generation 643 Max: 0.8356572857172884
Generation 643 Avg: 0.8342242059663433
Generation 644 Max: 0.8356572857172884
Generation 644 Avg: 0.8346207452735187
Generation 645 Max: 0.8356572857172884
Generation 645 Avg: 0.8335070478322144
Generation 646 Max: 0.8356572857172884
Generation 646 Avg: 0.8347590856171813
Generation 647 Max: 0.8356572857172884
Generation 647 Avg: 0.8345704544134988
Generation 648 Max: 0.8356572857172884
Generation 648 Avg: 0.8342995281431419
Generation 649 Max: 0.8356572857172884
Generation 649 Avg: 0.834883387490615
Generation 650 Max: 0.8356572857172884
Generation 650 Avg: 0.8348898350578089
Generation 651 Max: 0.8356572857172884
Generation 651 Avg: 0.834816993380081
Generation 652 Max: 0.8356572857172884
Generation 652 Avg: 0.83411

Generation 746 Max: 0.8356572857172884
Generation 746 Avg: 0.8344636594797072
Generation 747 Max: 0.8356572857172884
Generation 747 Avg: 0.8340655768351496
Generation 748 Max: 0.8356572857172884
Generation 748 Avg: 0.8346742127320061
Generation 749 Max: 0.8356572857172884
Generation 749 Avg: 0.8343622227624463
Generation 750 Max: 0.8356572857172884
Generation 750 Avg: 0.8347725466022307
Generation 751 Max: 0.8356572857172884
Generation 751 Avg: 0.8348686686247472
Generation 752 Max: 0.8356572857172884
Generation 752 Avg: 0.8349112431681092
Generation 753 Max: 0.8356572857172884
Generation 753 Avg: 0.8346804256117926
Generation 754 Max: 0.8356572857172884
Generation 754 Avg: 0.834544061967944
Generation 755 Max: 0.8356572857172884
Generation 755 Avg: 0.8337093224170867
Generation 756 Max: 0.8356572857172884
Generation 756 Avg: 0.8335538679329109
Generation 757 Max: 0.835773254993029
Generation 757 Avg: 0.8345359112644795
Generation 758 Max: 0.835773254993029
Generation 758 Avg: 0.833993

In [1]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import time
import math
import community as community_louvain
import networkx.algorithms.community as modularity
from networkx.algorithms.community.quality import modularity
from collections import defaultdict


G = nx.read_edgelist("C:/Users/gnssl/Downloads/facebook_combined.txt", 
                     create_using=nx.Graph(), nodetype=int)

partition_louvain = community_louvain.best_partition(G)
len_partition = len(partition_louvain)

population_size = 200
generations = 1000
crossover_prob = 1.0
mutation_prob = 0.05
local_optimization_frequency=5

min_partition_size = 5
max_partition_size = 15

def initialize_population():
    population = []
    for idx in range(population_size):
        if idx%2==0:
            partition_louvain = community_louvain.best_partition(G)
            partition = [partition_louvain[node] for node in G.nodes]
            population.append(partition)
        else:
            partition = [random.randint(0, max_partition_size) for _ in range(len(G.nodes))]
            population.append(partition)
    return population

def compute_fitness(partition):
    communities = {key: value for key, value in zip(partition_louvain.keys(), partition)}
    new_dict = defaultdict(list)

    for key, value in communities.items():
        new_dict[value].append(key)

    result_dict = dict(new_dict)
    fitness = nx.community.modularity(G, result_dict.values())
    return fitness


def select_parents(population, fitness_values):
    parent1, parent2 = random.sample(population, 2)
    if compute_fitness(parent1) > compute_fitness(parent2):
        return parent1
    else:
        return parent2

def crossover(parent1, parent2):
    size = len(parent1)
    p1, p2 = [0] * size, [0] * size

    # Select two random points in the chromosomes
    point1 = random.randint(0, size - 1)
    point2 = random.randint(0, size - 1)

    # Ensure the two points are not the same
    while point1 == point2:
        point2 = random.randint(0, size - 1)

    # Order the points
    if point1 > point2:
        point1, point2 = point2, point1

    # Copy the respective parts from the parents to the children
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return child1, child2

def mutate(individual, mutation_prob, max_partition_size):
    rand_tmp = random.random()
    rand_idx = random.randint(1,len(individual))
    if rand_tmp < mutation_prob:
        for i in range(0,len(individual), rand_idx):
            # Mutate the community assignment of the node to a random community
            individual[i] = random.randint(0, max_partition_size - 1)
    return individual

def partition_to_communities(partition):
    communities = {}
    for node, community_id in enumerate(partition):
        if community_id in communities:
            communities[community_id].add(node)
        else:
            communities[community_id] = {node}
    return list(communities.values())



def genetic_algorithm():
    population = initialize_population()
    max_fitness_values = []  # Store max fitness values for each generation

    for generation in range(generations):
        fitness_values = [compute_fitness(partition) for partition in population]
        max_fitness = max(fitness_values)
        max_fitness_values.append(max_fitness)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        print('Generation', generation + 1, 'Max:', max_fitness)
        print('Generation', generation + 1, 'Avg:', avg_fitness)

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parents(population, fitness_values)
            parent2 = select_parents(population, fitness_values)
            if random.random() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                
                child1 = mutate(child1, mutation_prob, max_partition_size)
                child2 = mutate(child2, mutation_prob, max_partition_size)
                
                min_fitness_index = min(range(len(fitness_values)), key=fitness_values.__getitem__)
                population[min_fitness_index] = child1
                second_min_fitness_index = min(range(len(fitness_values)), key=lambda x: fitness_values[x] if x != min_fitness_index else float('inf'))
                population[second_min_fitness_index] = child2
                
        
        if generation % local_optimization_frequency == 0:
            random_idx = random.randint(1,population_size//2)
            for idx in range(0, random_idx, population_size//random_idx):
                partition_louvain = community_louvain.best_partition(G)
                partition = [partition_louvain[node] for node in G.nodes]
                population[idx] = partition
       
    # Select the best partition from the final population
    best_partition = max(population, key=compute_fitness)
    return best_partition, max_fitness_values

start = time.time()
best_partition, max_fitness_values = genetic_algorithm()

best_communities = partition_to_communities(best_partition)
end = time.time()
print('time:', end - start)

print("Best Partition:", best_communities)

Generation 1 Max: 0.8357092336424166
Generation 1 Avg: 0.41719794520390396
Generation 2 Max: 0.8357092336424166
Generation 2 Avg: 0.4172162083136757
Generation 3 Max: 0.8357092336424166
Generation 3 Avg: 0.420891078965902
Generation 4 Max: 0.8357092336424166
Generation 4 Avg: 0.4249097429864355
Generation 5 Max: 0.8357092336424166
Generation 5 Avg: 0.42492270771631263
Generation 6 Max: 0.8357092336424166
Generation 6 Avg: 0.42872171886541915
Generation 7 Max: 0.8358598809169264
Generation 7 Avg: 0.42873403740252586
Generation 8 Max: 0.8358598809169264
Generation 8 Avg: 0.43709105256983316
Generation 9 Max: 0.8358598809169264
Generation 9 Avg: 0.43995976199994374
Generation 10 Max: 0.8358598809169264
Generation 10 Avg: 0.4451873029615019
Generation 11 Max: 0.8358598809169264
Generation 11 Avg: 0.4533391387967983
Generation 12 Max: 0.8357092336424166
Generation 12 Avg: 0.4781994165276307
Generation 13 Max: 0.8357092336424166
Generation 13 Avg: 0.48495813672416255
Generation 14 Max: 0.835

Generation 109 Max: 0.8358029743127893
Generation 109 Avg: 0.8255209892444694
Generation 110 Max: 0.8358029743127893
Generation 110 Avg: 0.8251097809957137
Generation 111 Max: 0.8358029743127893
Generation 111 Avg: 0.8257028409762367
Generation 112 Max: 0.8357092336424166
Generation 112 Avg: 0.826340560087442
Generation 113 Max: 0.8357092336424166
Generation 113 Avg: 0.8268029381477457
Generation 114 Max: 0.8357092336424166
Generation 114 Avg: 0.8268501735491285
Generation 115 Max: 0.8357092336424166
Generation 115 Avg: 0.8270394526860546
Generation 116 Max: 0.8357092336424166
Generation 116 Avg: 0.8264558579813968
Generation 117 Max: 0.8357092336424166
Generation 117 Avg: 0.8266329576206257
Generation 118 Max: 0.8357092336424166
Generation 118 Avg: 0.8256956245276776
Generation 119 Max: 0.8357092336424166
Generation 119 Avg: 0.8263673673353046
Generation 120 Max: 0.8357092336424166
Generation 120 Avg: 0.8275075598378347
Generation 121 Max: 0.8357092336424166
Generation 121 Avg: 0.8275

Generation 215 Max: 0.8357092336424166
Generation 215 Avg: 0.8333355576972491
Generation 216 Max: 0.8357092336424166
Generation 216 Avg: 0.8326590099123863
Generation 217 Max: 0.8357092336424166
Generation 217 Avg: 0.8334633844664323
Generation 218 Max: 0.8357092336424166
Generation 218 Avg: 0.8328177124674682
Generation 219 Max: 0.8357092336424166
Generation 219 Avg: 0.8335934221925355
Generation 220 Max: 0.8357092336424166
Generation 220 Avg: 0.8333523915968386
Generation 221 Max: 0.8357092336424166
Generation 221 Avg: 0.8322128231262298
Generation 222 Max: 0.8357092336424166
Generation 222 Avg: 0.8333298198921669
Generation 223 Max: 0.8357092336424166
Generation 223 Avg: 0.8336446751123247
Generation 224 Max: 0.8357092336424166
Generation 224 Avg: 0.8325952358454551
Generation 225 Max: 0.8357092336424166
Generation 225 Avg: 0.8333285719648004
Generation 226 Max: 0.8357092336424166
Generation 226 Avg: 0.8329689894185563
Generation 227 Max: 0.8357092336424166
Generation 227 Avg: 0.833

Generation 321 Max: 0.8357780244678215
Generation 321 Avg: 0.8341904701352104
Generation 322 Max: 0.8357780244678215
Generation 322 Avg: 0.8332442691835281
Generation 323 Max: 0.8357780244678215
Generation 323 Avg: 0.8347303072814073
Generation 324 Max: 0.8357780244678215
Generation 324 Avg: 0.8334549829129223
Generation 325 Max: 0.8357780244678215
Generation 325 Avg: 0.8343048544968092
Generation 326 Max: 0.8357780244678215
Generation 326 Avg: 0.8330715176418413
Generation 327 Max: 0.8357780244678215
Generation 327 Avg: 0.8342743645009416
Generation 328 Max: 0.8357780244678215
Generation 328 Avg: 0.8345525100999871
Generation 329 Max: 0.8357780244678215
Generation 329 Avg: 0.8332697010162832
Generation 330 Max: 0.8357780244678215
Generation 330 Avg: 0.831922492651361
Generation 331 Max: 0.8357780244678215
Generation 331 Avg: 0.8346580839366747
Generation 332 Max: 0.8357780244678215
Generation 332 Avg: 0.8334509536533861
Generation 333 Max: 0.8357780244678215
Generation 333 Avg: 0.8344

Generation 427 Max: 0.8357780244678215
Generation 427 Avg: 0.8348601887865469
Generation 428 Max: 0.8357780244678215
Generation 428 Avg: 0.8339613888949435
Generation 429 Max: 0.8357780244678215
Generation 429 Avg: 0.8347165306342343
Generation 430 Max: 0.8357780244678215
Generation 430 Avg: 0.8338585843573969
Generation 431 Max: 0.8357780244678215
Generation 431 Avg: 0.8348045899396693
Generation 432 Max: 0.8357780244678215
Generation 432 Avg: 0.8348612466417084
Generation 433 Max: 0.8357780244678215
Generation 433 Avg: 0.8348232528714757
Generation 434 Max: 0.8357780244678215
Generation 434 Avg: 0.8338385361915805
Generation 435 Max: 0.8357780244678215
Generation 435 Avg: 0.834763509148505
Generation 436 Max: 0.8357780244678215
Generation 436 Avg: 0.8338086883526404
Generation 437 Max: 0.8357780244678215
Generation 437 Avg: 0.834657946647051
Generation 438 Max: 0.8357780244678215
Generation 438 Avg: 0.8344216747711778
Generation 439 Max: 0.8357780244678215
Generation 439 Avg: 0.83470

Generation 533 Max: 0.8357780244678215
Generation 533 Avg: 0.83443656881536
Generation 534 Max: 0.8357780244678215
Generation 534 Avg: 0.8335668166610306
Generation 535 Max: 0.8357780244678215
Generation 535 Avg: 0.8346628258050565
Generation 536 Max: 0.8357780244678215
Generation 536 Avg: 0.8343288577972678
Generation 537 Max: 0.8357780244678215
Generation 537 Avg: 0.8329443296382317
Generation 538 Max: 0.8357780244678215
Generation 538 Avg: 0.8345209958083014
Generation 539 Max: 0.8357780244678215
Generation 539 Avg: 0.8348076771834588
Generation 540 Max: 0.8357780244678215
Generation 540 Avg: 0.8347455340494894
Generation 541 Max: 0.8357780244678215
Generation 541 Avg: 0.8345764008897661
Generation 542 Max: 0.8357780244678215
Generation 542 Avg: 0.8344942051206249
Generation 543 Max: 0.8357780244678215
Generation 543 Avg: 0.8347475183380236
Generation 544 Max: 0.8357780244678215
Generation 544 Avg: 0.8348700840347744
Generation 545 Max: 0.8357780244678215
Generation 545 Avg: 0.83475

Generation 639 Max: 0.8357934717782727
Generation 639 Avg: 0.8336878517656732
Generation 640 Max: 0.8357934717782727
Generation 640 Avg: 0.834551025066212
Generation 641 Max: 0.8357934717782727
Generation 641 Avg: 0.8345377642792619
Generation 642 Max: 0.8357780244678215
Generation 642 Avg: 0.8344071194760803
Generation 643 Max: 0.8357780244678215
Generation 643 Avg: 0.8337131693222283
Generation 644 Max: 0.8357780244678215
Generation 644 Avg: 0.8346008445158342
Generation 645 Max: 0.8357780244678215
Generation 645 Avg: 0.8345710084135283
Generation 646 Max: 0.8357780244678215
Generation 646 Avg: 0.8348941482762026
Generation 647 Max: 0.8358432842518722
Generation 647 Avg: 0.8346262334796406
Generation 648 Max: 0.8358432842518722
Generation 648 Avg: 0.8345824727302046
Generation 649 Max: 0.8358432842518722
Generation 649 Avg: 0.8348732976096285
Generation 650 Max: 0.8358432842518722
Generation 650 Avg: 0.8348039300488606
Generation 651 Max: 0.8358432842518722
Generation 651 Avg: 0.8347

Generation 745 Max: 0.8358149937258006
Generation 745 Avg: 0.8347029529064585
Generation 746 Max: 0.8358149937258006
Generation 746 Avg: 0.834837969995265
Generation 747 Max: 0.8358149937258006
Generation 747 Avg: 0.8345451977417175
Generation 748 Max: 0.8358149937258006
Generation 748 Avg: 0.8348800735097204
Generation 749 Max: 0.8358149937258006
Generation 749 Avg: 0.8331089480956958
Generation 750 Max: 0.8358149937258006
Generation 750 Avg: 0.8336926500960355
Generation 751 Max: 0.8358149937258006
Generation 751 Avg: 0.8349200847835891
Generation 752 Max: 0.8358149937258006
Generation 752 Avg: 0.8335026017418524
Generation 753 Max: 0.8358149937258006
Generation 753 Avg: 0.8340045475437057
Generation 754 Max: 0.8358149937258006
Generation 754 Avg: 0.8339499890098172
Generation 755 Max: 0.8358149937258006
Generation 755 Avg: 0.8345968128440404
Generation 756 Max: 0.8358149937258006
Generation 756 Avg: 0.8346474407043225
Generation 757 Max: 0.8358149937258006
Generation 757 Avg: 0.8343

Generation 851 Max: 0.8358149937258006
Generation 851 Avg: 0.8339513951946941
Generation 852 Max: 0.8358149937258006
Generation 852 Avg: 0.8349229556364557
Generation 853 Max: 0.8358149937258006
Generation 853 Avg: 0.8337217114475357
Generation 854 Max: 0.8358149937258006
Generation 854 Avg: 0.83377075543507
Generation 855 Max: 0.8358149937258006
Generation 855 Avg: 0.834617112118405
Generation 856 Max: 0.8358149937258006
Generation 856 Avg: 0.8326169864105711
Generation 857 Max: 0.8358149937258006
Generation 857 Avg: 0.8338650467924716
Generation 858 Max: 0.8358149937258006
Generation 858 Avg: 0.8341550059084666
Generation 859 Max: 0.8358149937258006
Generation 859 Avg: 0.8347352658936237
Generation 860 Max: 0.8358149937258006
Generation 860 Avg: 0.8345725312952115
Generation 861 Max: 0.8358149937258006
Generation 861 Avg: 0.8344905019477483
Generation 862 Max: 0.8358149937258006
Generation 862 Avg: 0.8329561821751795
Generation 863 Max: 0.8358149937258006
Generation 863 Avg: 0.833185

Generation 957 Max: 0.8357780244678215
Generation 957 Avg: 0.8349077854260485
Generation 958 Max: 0.8357780244678215
Generation 958 Avg: 0.8349412687000585
Generation 959 Max: 0.8357780244678215
Generation 959 Avg: 0.834935568244184
Generation 960 Max: 0.8357780244678215
Generation 960 Avg: 0.8347474362467715
Generation 961 Max: 0.8357780244678215
Generation 961 Avg: 0.8337894119772201
Generation 962 Max: 0.8357788567480127
Generation 962 Avg: 0.8346012925380607
Generation 963 Max: 0.8357788567480127
Generation 963 Avg: 0.8345392374254763
Generation 964 Max: 0.8357788567480127
Generation 964 Avg: 0.8338955949435887
Generation 965 Max: 0.8357788567480127
Generation 965 Avg: 0.83473403806776
Generation 966 Max: 0.8357788567480127
Generation 966 Avg: 0.8347065906009324
Generation 967 Max: 0.8357780244678215
Generation 967 Avg: 0.8344325133591325
Generation 968 Max: 0.8357780244678215
Generation 968 Avg: 0.8349219837003038
Generation 969 Max: 0.8357780244678215
Generation 969 Avg: 0.834353